In [1]:
import torch
import torch.nn as nn

class MessagePassingLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MessagePassingLayer, self).__init__()
        self.linear_message = nn.Linear(in_channels, out_channels)
        self.linear_update = nn.Linear(out_channels, out_channels)

    def forward(self, x, edge_index):
        # x: ノード特徴量 (num_nodes, in_channels)
        # edge_index: エッジの接続情報 (2, num_edges)
        
        # メッセージ生成
        row, col = edge_index
        messages = self.linear_message(x[col])  # 隣接ノードの特徴を取り出して線形変換
        
        # メッセージの集約（ここでは平均）
        aggr_messages = torch.zeros_like(x)  # 各ノードの初期化
        aggr_messages.index_add_(0, row, messages)  # メッセージを集約
        
        # 更新関数を適用
        return self.linear_update(aggr_messages)

# 使用例
if __name__ == "__main__":
    # ノード特徴量の次元（入力次元、出力次元）
    in_channels = 16
    out_channels = 32

    # ダミーデータ（ノード特徴量とエッジの接続情報）を作成
    num_nodes = 10
    x = torch.rand((num_nodes, in_channels))  # 10ノード分の特徴ベクトル
    edge_index = torch.tensor([[0, 1, 2, 3, 0],   # ノード間の接続を示すインデックス
                               [1, 2, 3, 4, 1]], dtype=torch.long)

    # メッセージパッシング層を適用
    message_passing_layer = MessagePassingLayer(in_channels, out_channels)
    x_updated = message_passing_layer(x, edge_index)

    print(x_updated)

RuntimeError: source tensor shape must match self tensor shape, excluding the specified dimension. Got self.shape = [10, 16] source.shape = [5, 32]